# Course project; Outer solar system
*Axel Eiman*

## Given simulation code:

In [2]:
import numpy as np
import bpy
import math
import mathutils

In [3]:
# Gravitational constant when the mass of the sun is 1.
G = 2.95912208286e-4

# Planet names and order
planets = ('Sun','Jupiter','Saturn','Uranus','Neptune','Pluto')

# The data below is obtained from here: https://ssd.jpl.nasa.gov/horizons.cgi

# Masses relative to the sun (the increased sun mass is to account for the inner planets)
masses = np.array([1.00000597682, 
                   0.000954786104043, 
                   0.000285583733151, 
                   0.0000437273164546, 
                   0.0000517759138449, 
                   6.571141277023631e-09])

# Positions of the planets in astronomical units (au) on September 5, 1994, at 0h00 GST.
positions = np.array([[0., 0., 0.],
                    [-3.502576677887171E+00, -4.111754751605156E+00,  9.546986420486078E-02],
                    [9.075323064717326E+00, -3.443060859273154E+00, -3.008002285860299E-01],
                    [8.309900066449559E+00, -1.782348877489204E+01, -1.738826162402036E-01],
                    [1.147049510166812E+01, -2.790203169301273E+01,  3.102324955757055E-01],
                    [-1.553841709421204E+01, -2.440295115792555E+01,  7.105854443660053E+00]])

# Velocities of the planets relative to the sun in au/day on September 5, 1994, at 0h00 GST.
velocities = np.array([[0., 0., 0.],
                    [5.647185685991568E-03, -4.540768024044625E-03, -1.077097723549840E-04],
                    [1.677252496875353E-03,  5.205044578906008E-03, -1.577215019146763E-04],
                    [3.535508197097127E-03,  1.479452678720917E-03, -4.019422185567764E-05],
                    [2.882592399188369E-03,  1.211095412047072E-03, -9.118527716949448E-05],
                    [2.754640676017983E-03, -2.105690992946069E-03, -5.607958889969929E-04]])

# Compute total linear momentum
ptot = (masses[:,np.newaxis]*velocities).sum(axis=0)

# Recompute velocities relative to the center of mass
velocities -= ptot/masses.sum()

# Linear momenta of the planets: p = m*v
momenta = masses[:,np.newaxis]*velocities

# Function for Newtonian acceleration field
def acc(x, masses = masses, G = G):
    N = masses.shape[0]
    d = x.shape[-1]
    dx_pairs = x[:, np.newaxis] - x[np.newaxis, :]
    msq_pairs = masses[:, np.newaxis]*masses[np.newaxis, :]
    
    # Remove self-self interactions
    dx_pairs = np.delete(dx_pairs.reshape((N*N,d)),slice(None,None,N+1), axis = 0).reshape((N,N-1,d))
    msq_pairs = np.delete(msq_pairs.reshape((N*N)),slice(None,None,N+1), axis = 0).reshape((N,N-1))
    
    # Compute pairwise distances
    dist_pairs = np.sqrt((dx_pairs**2).sum(axis=-1))
    
    # Compute the gravitational force using Newton's law
    forces = -G*(dx_pairs*msq_pairs[:,:,np.newaxis]/dist_pairs[:,:,np.newaxis]**3).sum(axis=1)
    
    # Return accelerations
    return forces/masses[:,np.newaxis]

# Select time step and total integration time (measured in days)
h = 100 # Time stepsize in days
totaltime = 100*365 # Total simulation time in days

# Preallocate output vectors at each step
t_out = np.arange(0.,totaltime,h)
x_out = np.zeros(t_out.shape + positions.shape, dtype=float)
x_out[0,:,:] = positions
v_out = np.zeros_like(x_out)
v_out[0,:,:] = velocities

C:\Users\Admin\AppData\Local\Temp\tmpvc3ozlef\kernel.py:44: DeprecationWarning: There is no current event loop
  loop = asyncio.get_event_loop()


## My code

Because I was interested, I worked from this notebook and started scripting more than was really necessary as you can see below. Quite soon I realized that it would be easier to do more "manually". Some of the code is not necessary, but it is how I got started before doing more modifications within blender itself. Since it was already here I thought I might as well keep it in the report. 

In [ ]:
myobj = bpy.data.objects['Cube']

# Delete object 
bpy.data.objects.remove(myobj, do_unlink=True)

# Adding the planet meshes 
for i, name in enumerate(planets):
    bpy.ops.mesh.primitive_uv_sphere_add(location=positions[i])
    bpy.context.object.name = name
    bpy.context.object.data.name = name
    bpy.context.object.keyframe_insert(data_path='location')


# Set up the animation time etc.

nframes = 365
bpy.context.scene.frame_start = 0
bpy.context.scene.frame_end = nframes
nlast = bpy.context.scene.frame_end


In [4]:

# Use Symplectic Euler method for integration
for i, (x0, x1, v0, v1) in enumerate(zip(x_out[:-1],x_out[1:],v_out[:-1],v_out[1:]), 1):
    x1[:,:] = x0 + h*v0
    v1[:,:] = v0 + h*acc(x1)

    x_out[i,:,:] = x1
    v_out[i,:,:] = v1
    
    bpy.context.scene.frame_set(i)
    for j, name in enumerate(planets):
        c_obj = bpy.data.objects[name]
        c_obj.location = x_out[i,j]
        c_obj.keyframe_insert(data_path='location')
# -------------------------

# print(x_out[0, 1])

[-3.50257668 -4.11175475  0.09546986]


# My solution

As seen in the code above, I created and named the planets with a script. The textures I found on the page linked on canvas were not possible to open at all or use in blender, so I looked elsewhere for the planet texture images. For the rings around the planets I made a mesh and shaded it with generated noise for both colors and transparency to make them look somewhat realistic.

For the lighting from the sun I made the material of the mesh emit light. To make it a bit more interesting, I controlled the color of the emitted light with the same texture as the sphere itself, although mellowed out somewhat to make it look more natural. 

For the camera animation I wanted to start with an overview of the planets, and then get a shot following Jupiter around before heading out to an overhead view of the system. This was done by adding position keyframes for the camera, and changing the influence level between which object the camera is locked onto when I wanted it to point to Jupiter or the sun.

The detail of the rendered animation is unfortunately not as good as I had ambition for. When I first tried to render I was quite surprised at just how long every single frame took to render on my computer. Trying to reduce rendering time to a reasonable level, I removed things like the subdivision surface modifiers on the spheres, so they look quite boxy. I also reduced the resolution and output quality way below where I would like it to be. 

Trying to optimize the rendering time took a lot longer than I expected, so I did not have time to add some things I wanted to (some of which likely would make the rendering even slower). I wanted to add a fire simulation for the sun, to make it so shadows are cast on the rings, as well as add a fancier background, rotations, tilts, and sizes to make the system a bit more realistic. Unfortunately I did not find the time for this and solving my rendering issues, so opted to stay safe and stick to a simpler animation. 

In [11]:
from IPython.display import Video
Video('outputs/0000-0364.mp4', embed=True)